<a href="https://colab.research.google.com/github/lmquann/DoAnPTDLPython2022/blob/main/DOAN_ThongKeSuyDien.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/lmquann/leminhquan/main/Diet_R.csv")
df.sample(10)

,Person,gender,Age,Height,pre.weight,Diet,weight6weeks
72,73,1,40,179,79,3,74.5
41,63,0,35,171,72,3,64.4
75,76,1,28,164,85,3,79.7
33,55,0,22,169,65,3,58.2
62,48,1,40,171,79,2,72.9
31,53,0,35,169,62,3,56.4
76,77,1,40,167,87,3,77.8
74,75,1,49,177,84,3,79.9
9,8,0,28,176,69,1,60.5
46,16,1,31,158,72,1,70.9


Xóa các dữ liệu trống:

In [3]:
#Chuyển đổi các ô trống sang NaN
df.replace(" ", np.nan, inplace=True) 
#Xóa các hàng có giá trị NaN
df.dropna(inplace=True) 
df.sample(10)

,Person,gender,Age,Height,pre.weight,Diet,weight6weeks
38,60,0,58,141,70,3,63.0
23,34,0,54,173,63,2,59.4
49,19,1,43,162,80,1,71.0
71,72,1,26,179,78,3,69.4
48,18,1,50,160,78,1,73.9
65,51,1,31,182,80,2,75.7
57,43,1,54,196,75,2,69.2
60,46,1,37,194,78,2,76.3
17,28,0,37,172,58,2,56.0
44,66,0,41,157,76,3,71.0


# **Phát biểu câu hỏi nghiên cứu và tiến hành phân tích**

**Ignore diet and test to see if weight has been lost(Bỏ qua chế độ ăn kiêng và kiểm tra xem cân nặng có giảm hay không?)**

Phát biểu giả thiết:

+ $H_0$ : Cân nặng trước và sau khi kiểm tra bằng nhau.

+ $H_a$ : Cân nặng trước và sau khi kiểm tra có sự khác nhau. 

In [4]:
ttest_score, p = st.ttest_rel(df["pre.weight"], df["weight6weeks"])
print("Stat =", ttest_score, "P_value =", p)
if p > 0.05:
    print("không có bằng chứng để bác bỏ giả thiết H0")
else:
    print("có bằng chứng để bác bỏ giả thiết H0, điều này ngụ ý chấp nhận giả thiết Ha")

Stat = 13.728486740903172 P_value = 3.7220652146643845e-22
có bằng chứng để bác bỏ giả thiết H0, điều này ngụ ý chấp nhận giả thiết Ha


**=> Cân nặng trước và sau khi kiểm tra có sự khác nhau**

**Remove weight lost and get students to calculate it using before/after weights(Loại bỏ trọng lượng đã giảm và tính toán nó bằng cách sử dụng trọng lượng trước / sau)**

In [5]:
df["Weightlost"] = (df["pre.weight"] - df["weight6weeks"]).values
df.sample(10)

,Person,gender,Age,Height,pre.weight,Diet,weight6weeks,Weightlost
27,38,0,37,167,68,2,63.3,4.7
14,13,0,41,163,72,1,68.4,3.6
37,59,0,45,155,69,3,61.8,7.2
26,37,0,16,165,68,2,63.8,4.2
44,66,0,41,157,76,3,71.0,5.0
5,4,0,33,171,64,1,61.1,2.9
3,2,0,46,192,60,1,54.0,6.0
76,77,1,40,167,87,3,77.8,9.2
56,42,1,38,199,75,2,72.6,2.4
10,9,0,28,165,70,1,68.1,1.9


**Summary statistics by diet (phân nhóm giá trị weightlost theo diet)**

In [6]:
w1 = df[(df["Diet"] == 1)]["Weightlost"]
w2 = df[(df["Diet"] == 2)]["Weightlost"]
w3 = df[(df["Diet"] == 3)]["Weightlost"]

Trước khi kiểm định ANOVA ta cần tiến hành kiểm tra tập dữ liệu thỏa mãn 3 tính chất sau: 
+ Tất cả các nhóm đều độc lập với nhau
+ Dữ liệu của mỗi nhóm đều tuân theo phân phối chuẩn. .
+ Phương sai (variance) của các nhóm bằng nhau. 


**Which diet was best for losing weight?(Chế độ ăn kiêng nào là tốt nhất để giảm cân?)**

**Kiểm tra phân phối chuẩn:**

Phát biểu giả thiết:

+ $H_0$ : Các mẫu tuân theo phân phối chuẩn.

+ $H_a$ : Các mẫu không tuân theo phân phối chuẩn. 

In [7]:
#Kiểm tra Diet1 với mức ý nghĩa 0.05: 
stat, p = st.shapiro(w1) 
print("Stat =", stat, "\n", "p value =", p)
if p > 0.05:
    print("không có bằng chứng để bác bỏ giả thiết H0")
else:
    print("có bằng chứng để bác bỏ giả thiết H0, điều này ngụ ý chấp nhận giả thiết Ha")

Stat = 0.9255314469337463 
 p value = 0.07748671621084213
không có bằng chứng để bác bỏ giả thiết H0


In [8]:
#Kiểm tra Diet2 với mức ý nghĩa 0.05: 
stat, p = st.shapiro(w2) 
print("Stat =", stat, "\n", "p value =", p)
if p > 0.05:
    print("không có bằng chứng để bác bỏ giả thiết H0")
else:
    print("có bằng chứng để bác bỏ giả thiết H0, điều này ngụ ý chấp nhận giả thiết Ha")

Stat = 0.9793620705604553 
 p value = 0.8721737861633301
không có bằng chứng để bác bỏ giả thiết H0


In [9]:
#Kiểm tra Diet3 với mức ý nghĩa 0.05: 
stat, p = st.shapiro(w3) 
print("Stat =", stat, "\n", "p value =", p)
if p > 0.05:
    print("không có bằng chứng để bác bỏ giả thiết H0")
else:
    print("có bằng chứng để bác bỏ giả thiết H0, điều này ngụ ý chấp nhận giả thiết Ha")

Stat = 0.9601252675056458 
 p value = 0.37203603982925415
không có bằng chứng để bác bỏ giả thiết H0


=> Các mẫu đều tuân theo phân phối chuẩn

**Kiểm tra tính đồng nhất phương sai:**

Phát biểu giả thuyết: 

+ $H_0$: Các mẫu có cùng phương sai. 
  
 
+ $H_a$: Có sự khác biệt về phương sai của các mẫu. 

In [10]:
#Kiểm tra tính đồng nhất phương sai với mức ý nghĩa 0.05:
stat, p = st.levene(w1, w2, w3) 
print("Statistic =", stat, "\n", "p value =", p)
if p > 0.05:
  print("không có bằng chứng để bác bỏ giả thiết H0")
else:
  print("có bằng chứng để bác bỏ giả thiết H0, điều này ngụ ý chấp nhận giả thiết Ha")

Statistic = 0.46290764582507116 
 p value = 0.6312856401117967
không có bằng chứng để bác bỏ giả thiết H0


=> Các mẫu có sự đồng nhất về phương sai

Vì các mẫu độc lập, tuân theo phân phối chuẩn và có sự đồng nhất về phương sai giữa các mẫu nên ta có thể tiến hành kiểm định ANOVA

**Kiểm tra tính đồng nhất về giá trị trung bình:**

Phát biểu giả thuyết:

+ $H_0$: Các mẫu có giá trị trung bình bằng nhau. 

+ $H_a$: Có sự khác biệt về giá trị trung bình của các mẫu.

In [11]:
stat, p = st.f_oneway(w1, w2, w3) 
print("Statistic =", stat, "\n", "p value =", p)
if p > 0.05:
  print("không có bằng chứng để bác bỏ giả thiết H0")
else:
  print("có bằng chứng để bác bỏ giả thiết H0, điều này ngụ ý chấp nhận giả thiết Ha")

Statistic = 5.383104474859926 
 p value = 0.006595853108669741
có bằng chứng để bác bỏ giả thiết H0, điều này ngụ ý chấp nhận giả thiết Ha


**=> Điều này chứng tỏ chế độ ăn kiêng có ảnh hưởng đến việc giảm cân.**

In [12]:
d = {}
d["D1"] = np.mean(w1)
d["D2"] = np.mean(w2)
d["D3"] = np.mean(w3)
d0 = pd.DataFrame(d, index=[''])
d0

,D1,D2,D3
,3.3,3.268,5.148148


**=> Chế độ ăn kiêng hiệu quả nhất là nhóm Diet 3**

**Are there gender differences for weight loss?(Có sự khác biệt về giới tính đối với việc giảm cân?)**

In [13]:
#Phân nhóm giá trị Weightlost theo gender: 
g0 = df[(df["gender"] == '0')]["Weightlost"]
g1 = df[(df["gender"] == '1')]["Weightlost"]

Phát biểu giả thiết:

+ $H_0$ : Chỉ số cân nặng giảm được của nam và nữ bằng nhau.

+ $H_a$ : Chỉ số cân nặng giảm được của nam và nữ khác nhau. 

In [14]:
ttest_score, p = st.ttest_ind(g0, g1, equal_var=False)
print("Stat =", ttest_score, "P_value =", p)
if p > 0.05:
    print("không có bằng chứng để bác bỏ giả thiết H0")
else:
    print("có bằng chứng để bác bỏ giả thiết H0, điều này ngụ ý chấp nhận giả thiết Ha")

Stat = -0.20909666699582563 P_value = 0.8349905694773132
không có bằng chứng để bác bỏ giả thiết H0


**=> Giới tính không có ảnh hưởng đến việc giảm cân**